# Fudge Wrapper

The Fudge() function wraps an Apath object in a Fudge interface allowing access to branches and attributes through a simple, familiar path-ish syntax.

    fu = Fudge(apath)
    
The fluent methods of fudge emulate the _fudge_ minilanguage which is a subset of regular filepath syntax.

Fudge path language:
    `/this/that/other.type/item@attribute`

Fudge syntax emulation:

    fu['/this/that/other'].type['/item']['@attribute']
    
The fudgepath language can be evaluated by an interpreter to get the same result:

    `fu/'this/that/other.type/item@attribute'`
    
Children and attributes are mapped using types from the cosmos system of the access path. Some types transform data.

    fu['/this'].transform

Some types create a map:
    
    fu['/that'].map['/other']['@somefield']
    
As an example, a field from an fstab file can be accessed like this:

    fu = Fudge(fstab_apath)
    fu.fstab['/'+some_entryname]['@vfstype']


## API namespace

A Fudge object represents the namespace of the domain it wraps by generating an object interface for it. 

Items accessed from the Dcel `__getitem__()` interface are available via the Fudge `__getitem__()` interface when the item key is prefixed with '/'.

Attributes accessed from the `Dcel.map` are available via the Fudge `__getitem__()` interface when the item key is prefixed with '@'.

Type functions from `APath.cosm['types']` are available via the Fudge `__getattr__()` interface and create an APath branch with a Dcel with a _producer with default (uninitialized) arguments._

Arguments are added to the producer via the Fudge `__call__()` interface to create a branch with initialized arguments. Each invocation of `__call_()` results in a separate access path branch.


## R Value

The Fudge class cannot provide most other interfaces such as pyfilesystem because of the override of the `__getattr__()` method.

To get data from a fudge use its string representation:

    str(fu)
    or fu.__str__()
    
Or take the Apath out of the wrapper:

    Apath(fu)

### Lazy Evaluation

See also Dcel and APath.

The Fudge methods set up a chain of Dcels via Access Paths. The chain is not evaluated until the R Value is requested.

Dcels and APaths provide R Values via their file system, getitem, getattr, string and value interfaces.

The methods used to construct the chain are strictly initialization methods which set Dcel production parameters.

Access Path lookup may succeed even though its Dcel R Value request may fail. Therefor some APaths may not map to data and should not be represented in a directory listing.


## Fudgestar (\*) explained

The fudgestar glob works a little differently depending on whether it is addressed as a leaf

    a/b/*

or a branch.

    a/b/*/
    
In the case of a leaf, the fudge star does nothing but return the same filelist as its parent segment.

    a/b/ === a/b/*
    
But if the fudgestar has a trailing slash or a following segment, those must be evaluated for every item in the list.

    a/b/*/ == [ ea/ for ea in a/b/ ]
    
    
### Glob Filetype

In addition to dir and file, there needs to be a glob filetype to support the fudgestar behavior.

    fudge._apath._isglob

    

In [1]:
from APath import APath, APathWrapper
from os.path import basename
from urllib.parse import urlparse


from DictFS import DictFS
from MulticelSeqFS import MulticelSeqFS
from Dcel import Dcel

def fudgeglob(apath,expr):
    dcel_list = list()
    res_dcel = Dcel(dcel_list,
                    service_class=MulticelSeqFS
                   )
    res = Fudge(apath.spawn(expr,
                            res_dcel))
    return res


class Fudge(APathWrapper):
    
    def __init__(self,source):
        # default string to 'file'
        srctype = type(source)
        if srctype is str:
            service_class = 'file'
            apath = APath(source, 
                          service_class
                         )
        if issubclass(srctype,APath):
            apath = source
        if issubclass(srctype,(dict, list)):
            service_class = 'dict'
            apath = APath(source, 
                          service_class
                         )
        # APathWrapper initializes
        #   self._apath
        super().__init__(apath)
        
    def _getdir(self):
        if self._apath.isdir():
            return self
        else:
            try:
                addr = self._apath.target.address
                typename = basename(addr)
                res = self.__getattr__(typename)
                if res._apath.isdir():
                    return res
            except:
                raise
        raise NotADirectoryError
    
    def __getitem__(self, item: str):
        """
        Return child from directory
        surface of the domain,
        wrapped in fudge.
        
        If the parent is a glob,
        then the item returned is a
        list of matching items for
        each item in the glob result.
        
        If the item is a glob, then
        spawn a new target with fudge
        """
        
        if item == '.':
            return self
        
        if hasattr(self._apath,'_isglob'):
            res = Fudge([ea[item]
                          for ea in self
                         ])
            res._apath._isglob = True
            return res
        
        if item == '*':
            res = Fudge([ea for ea in self])
            res._apath._isglob = True
            return res
        
        directory = self._getdir()

        if item == '*':
            dcel = directory._apath.target
            items = [dcel[ea]
                     for ea in dcel.listdir()]
            globcel = Dcel(items,
                           service_class=MulticelSeqFS)
            globapath = self._apath._spawn('*',globcel)
            globapath._isglob = True
            return Fudge(globapath)
        
        return Fudge(directory._apath.getchild(item))
    
    def __setitem__(self, pathstr, value):
        target = self[pathstr]._apath.target
        target.value = value
        print(f"Fudge::__setitem__() target.value type: {type(target.value)}")
        target.value.flush()
        
    def __getattr__(self, typename):
        """
        Return a cosmos 'type'
        mutation of the _apath.
        """
        #1. lookup cosm/type
        #2. mutate the apath
        
        #typefn = self._apath.cosm['types'][typename]
        mutation = self._apath.command(dict, typename, ~(self._apath.target))
        return Fudge(mutation)
    
    def __call__(self, *args, **kwargs):
        """
        Pass domain through a function.
        """
        return Fudge(self._apath)
    
    #def __truediv__(self, item):
        #!!! this is overridden below
    
    def __matmul__(self, item):
        return self.__getattr__(item)
    
    def __mul__(self, item):
        return self.__getattr__(item)
    
    def __gt__(self, other):
        if other == 'hello':
            return 'hi'
        return Fudge(self._apath)
    
    def __eq__(self, other):
        if type(other) is str:
            return str(self) == other
        else:
            raise TypeError(other)
    
    def __str__(self):
        return str(self._apath.target)

    def __iter__(self):
        try:
            directory = self._getdir()
            if hasattr(self._apath,'_isglob'):
                return iter([ea for ea in directory._apath.listdir()])
            return iter([self[ea] for ea in directory._apath.listdir()])
        except:
            return iter([self])
        
            
# Fudge pathwalk

def splitexists(fudgeob, seg, c='.'):
    parts = seg.split(c)
    i = 0
    matchi = -1
    match = ''
    trypart = ''
    fudgeob = fudgeob._getdir()
    for part in parts:
        if trypart == '':
            trypart = part
        else:
            trypart += c + part
        if fudgeob._apath.exists(trypart):
            matchi = i
            match = trypart
        i += 1
    return (match,parts[matchi+1:])

def fudgedots(nextfudge,suffixes):
    for suf in suffixes:
        nextfudge = nextfudge.__getattr__(suf)
    return nextfudge

def fudgedot(fudgeob,path):
    if hasattr(fudgeob._apath,'_isglob'):
        array = [fudgedot(each,path)
                 for each in fudgeob]
        res = Fudge(array)
        res._apath._isglob = True
        return res
    exists,segs = splitexists(fudgeob,path,'.')
    nextfudge = fudgeob
    if exists != '':
        nextfudge = fudgeob[exists]
    if segs == []:
        return nextfudge
    else:
        return fudgedots(nextfudge, segs)

def fudgestar(fudgeob,segs):
    ar = [ fudgesegs(fudgeob[name],segs)._apath.target.value
            for name in fudgeob ] 
    return Fudge(ar)

def fudgesegs(nextfudge,segs):
    for seg in segs:
        if seg == '*':
            nextfudge = nextfudge[seg]
            continue
        nextfudge = fudgedot(nextfudge,seg)
    return nextfudge

def fudgewalk(fudgeob,path):
    try:
        return fudgeob[path]
    except KeyError:
        exists,segs = splitexists(fudgeob,path,'/')
        if segs == []:
            return fudgedot(fudgeob,path)
        if exists != '':
            nextfudge = fudgeob[exists]
            return fudgesegs(nextfudge, segs)
        else:
            return fudgesegs(fudgeob, segs)

Fudge.__truediv__ = fudgewalk

### SETUP TEST ENVIRONMENT

In [2]:
from blackstrap import BlackstrapFS

BlackstrapFS.initHost('localhost')
BlackstrapFS.addShare('fs/fs','fs')


blackstrap.BlackstrapFS

In [3]:
# file system root

fu_root = Fudge('file://fs.localhost')

In [4]:
# TODO: FIXME: accessing root is somehow required
# before using a '.command' fudge as in the next jupyter cell
# otherwise the '.command' lookup fails.
# Why?
rootdir = fu_root/'/'

### TEST .CSKVP PARSER

In [27]:
mntops = fu_root/'@/etc/fstab/1/mntopts.cskvp'

creating Dcel with formula: <function hiena_mp at 0x7f4018265040>, args: [<Dcel.Dcel object at 0x7f40183859a0>, <Dcel.DcelReference object at 0x7f4001e80070>], kwargs: {}
Dcel service wrapper: <Dcel.Dcel object at 0x7f4001e6e670>
APath wrapper: <APath.APath object at 0x7f4001dba670>
branch key: <APath.APath object at 0x7f4018385820>
creating Dcel with formula: <function hiena_mp at 0x7f4018265040>, args: [<Dcel.Dcel object at 0x7f40183859a0>, <Dcel.DcelReference object at 0x7f401802d430>], kwargs: {}
Dcel service wrapper: <Dcel.Dcel object at 0x7f4018320700>
APath wrapper: <APath.APath object at 0x7f4018320f40>
branch key: <APath.APath object at 0x7f4018385820>
creating Dcel with formula: <function hiena_mp at 0x7f4018265040>, args: [<Dcel.Dcel object at 0x7f4001bc8e80>, <Dcel.DcelReference object at 0x7f4001bc8ca0>], kwargs: {}
Dcel service wrapper: <Dcel.Dcel object at 0x7f4001bc8d60>
APath wrapper: <APath.APath object at 0x7f4001bc8eb0>
branch key: <APath.APath object at 0x7f4018149

In [39]:
print(repr(mntops))
print('---')
for ea in mntops:
    print(f"{repr(ea)} {ea._apath.getinfo().name}")

---
<__main__.Fudge object at 0x7f4001aa5670> user
<__main__.Fudge object at 0x7f40019eac70> shortid
<__main__.Fudge object at 0x7f40019ea5b0> idcard


In [42]:
print(mntops/'idcard')

localuser


### TEST FUDGE GLOB OUTPUT

In [5]:
dirlist = fu_root/'*'

In [15]:
for ea in dirlist:
    print(f"{repr(ea)} {ea._apath.getinfo()}")

DictFS: fsdict type is <class 'list'>, path is /. Returning fsdict without lookup.
DictFS: fsdict type is <class 'list'>, path is /. Returning fsdict without lookup.
<__main__.Fudge object at 0x7f40183430d0> <dir '.cloud'>
<__main__.Fudge object at 0x7f40183315e0> <dir '..@'>
<__main__.Fudge object at 0x7f4018324940> <dir '.cosm'>
<__main__.Fudge object at 0x7f4018324340> <dir 'fruit'>
<__main__.Fudge object at 0x7f40183cfe80> <dir 'numbers'>
<__main__.Fudge object at 0x7f40183cf730> <dir 'boats'>
<__main__.Fudge object at 0x7f40183cf700> <dir '@'>


In [24]:
schemelist = fu_root/'@/etc/fstabtest.fstab/*/spec.url/scheme'

creating Dcel with formula: <function hiena_mp at 0x7f4018265040>, args: [<Dcel.Dcel object at 0x7f40183859a0>, <Dcel.DcelReference object at 0x7f40183c4190>], kwargs: {}
Dcel service wrapper: <Dcel.Dcel object at 0x7f40026e6ca0>
APath wrapper: <APath.APath object at 0x7f4001c4de50>
branch key: <APath.APath object at 0x7f40026e0bb0>
DictFS: fsdict type is <class 'list'>, path is /. Returning fsdict without lookup.
DictFS: fsdict type is <class 'list'>, path is /. Returning fsdict without lookup.
creating Dcel with formula: <function urlparse_wrapper at 0x7f401822d430>, args: (<Dcel.DcelReference object at 0x7f4018370e50>,), kwargs: {}
Dcel service wrapper: <Dcel.Dcel object at 0x7f4018370370>
APath wrapper: <APath.APath object at 0x7f4018185160>
branch key: <APath.APath object at 0x7f40183c8a60>
creating Dcel with formula: <function urlparse_wrapper at 0x7f401822d430>, args: (<Dcel.DcelReference object at 0x7f4018185220>,), kwargs: {}
Dcel service wrapper: <Dcel.Dcel object at 0x7f4018

In [25]:
for ea in schemelist:
    print(ea)

DictFS: fsdict type is <class 'list'>, path is /. Returning fsdict without lookup.
DictFS: fsdict type is <class 'list'>, path is /. Returning fsdict without lookup.
cloud
file
file
file


### TEST URL PARSER

In [5]:
fstab = fu_root/'@/etc/fstab'

In [6]:
print(fstab/'1/spec.url/scheme')

creating Dcel with formula: <function hiena_mp at 0x7f4b11eecee0>, args: [<Dcel.Dcel object at 0x7f4b11f61f10>, <Dcel.DcelReference object at 0x7f4b11f54460>], kwargs: {}
Dcel service wrapper: <Dcel.Dcel object at 0x7f4b11f61580>
APath wrapper: <APath.APath object at 0x7f4b11f61610>
branch key: <APath.APath object at 0x7f4b127ab910>
creating Dcel with formula: <function hiena_mp at 0x7f4b11eecee0>, args: [<Dcel.Dcel object at 0x7f4b11f61f10>, <Dcel.DcelReference object at 0x7f4b11f41880>], kwargs: {}
Dcel service wrapper: <Dcel.Dcel object at 0x7f4b11f41220>
APath wrapper: <APath.APath object at 0x7f4b12ff1d90>
branch key: <APath.APath object at 0x7f4b127ab910>
creating Dcel with formula: <function hiena_mp at 0x7f4b11eecee0>, args: [<Dcel.Dcel object at 0x7f4b11f61f10>, <Dcel.DcelReference object at 0x7f4b11f43820>], kwargs: {}
Dcel service wrapper: <Dcel.Dcel object at 0x7f4b11f43880>
APath wrapper: <APath.APath object at 0x7f4b11f43400>
branch key: <APath.APath object at 0x7f4b127ab

In [7]:
url = fstab['1']['spec'].url

creating Dcel with formula: <function hiena_mp at 0x7f4b11eecee0>, args: [<Dcel.Dcel object at 0x7f4b11f61f10>, <Dcel.DcelReference object at 0x7f4b11f41700>], kwargs: {}
Dcel service wrapper: <Dcel.Dcel object at 0x7f4b11f416d0>
APath wrapper: <APath.APath object at 0x7f4b11f61eb0>
branch key: <APath.APath object at 0x7f4b127ab910>
creating Dcel with formula: <function urlparse_wrapper at 0x7f4b11e4e310>, args: (<Dcel.DcelReference object at 0x7f4b11f41370>,), kwargs: {}
Dcel service wrapper: <Dcel.Dcel object at 0x7f4b11f76880>
APath wrapper: <APath.APath object at 0x7f4b11f765b0>
branch key: <APath.APath object at 0x7f4b11f76be0>


In [20]:
print(url._apath.target.service)

In [7]:
print(repr(url._apath.target))
# same as Dcel service wrapper, above.

print(repr(url._apath.target.transformation_history))
# address o fwrapper is supposed to be previous dcel.

transource = url._apath.target.transformation_history

print(str(transource.args[0]))

file://fs2.localhost/


In [8]:
print(url)

{'scheme': 'file', 'netloc': 'fs2.localhost', 'path': '/', 'params': '', 'query': '', 'fragment': ''}


### Develop multivalue update from JSON

In [5]:
path = '@/etc/fstabtest.fstab'
fstab = fu_root/(path+'/')

In [44]:
# create JSON
from json import dumps, loads
from DcelJSONEncoder import DcelJSONEncoder
multiValue = dumps(fstab._apath.target,cls=DcelJSONEncoder)

In [46]:
# create list of (URL,VALUE)
jsonMultiValue = loads(multiValue)

def urlListFromDict(path, ob) -> list:
    dirlist = list()
    if type(ob) is dict:
        for each in ob:
            dirlist = dirlist + urlListFromDict(f"{path}/{each}", ob[each])
    else:
        return [(path, ob)]
    return dirlist
                
updates = urlListFromDict(path, jsonMultiValue)
for url,val in updates:
    print(f"{url} = {val}")

@/etc/fstabtest.fstab/1/spec = cloud://burner.localhost
@/etc/fstabtest.fstab/1/file = {cburnuser}/example/
@/etc/fstabtest.fstab/1/vfstype = cburnfs
@/etc/fstabtest.fstab/1/mntopts = user,shortid=9mm,idcard=smithandwesson.id.pub
@/etc/fstabtest.fstab/1/freq = 0
@/etc/fstabtest.fstab/1/passno = 0
@/etc/fstabtest.fstab/2/spec = file://fs.localhost
@/etc/fstabtest.fstab/2/file = {cburnuser}/example/
@/etc/fstabtest.fstab/2/vfstype = cburnfs
@/etc/fstabtest.fstab/2/mntopts = user,shortid=fs,idcard=localuser
@/etc/fstabtest.fstab/2/freq = 0
@/etc/fstabtest.fstab/2/passno = 0
@/etc/fstabtest.fstab/3/spec = file://fs3.localhost/
@/etc/fstabtest.fstab/3/file = {cburnuser}/example/
@/etc/fstabtest.fstab/3/vfstype = cburnfs
@/etc/fstabtest.fstab/3/mntopts = user,shortid=f3,idcard=localuser
@/etc/fstabtest.fstab/3/freq = 0
@/etc/fstabtest.fstab/3/passno = 0
@/etc/fstabtest.fstab/4/spec = file://cburnwebui.localhost/
@/etc/fstabtest.fstab/4/file = {cburnuser}/example/
@/etc/fstabtest.fstab/4/vfst

### Test writing multiple values and flushing to storage.

Following is the working way to *write* to a Fudge. Use the whole URL. This method parses the Fudge completely before writing to it. It's the only predictable method currently.

In [7]:
# WORKS: so here's what works: re-parsing each time a value needs to change
#        will make sure that the data gets mapped to the correct bytes in storage.

mntops = fu_root/'@/etc/fstabtest.fstab/1/mntopts.cskvp'
mntops['shortid'] = 'PF2'

# re-parse...

mntops = fu_root/'@/etc/fstabtest.fstab/1/mntopts.cskvp'
mntops['idcard'] = 'dannyboy.id'

# re-parse...

spec = fu_root/'@/etc/fstabtest.fstab/1/spec'
spec['.'] = "git://fs2.localhost"


In [9]:
# shorthand...

(fu_root/'@/etc/fstabtest.fstab/1/mntopts.cskvp/shortid')['.'] = '9mm'
(fu_root/'@/etc/fstabtest.fstab/1/mntopts.cskvp/idcard')['.'] = 'smithandwesson.id.pub'
(fu_root/'@/etc/fstabtest.fstab/1/spec')['.'] = 'cloud://burner.localhost'


### Developing flush()

In [5]:
mntopts = fu_root/'@/etc/fstabtest.fstab/1/mntopts.cskvp'

In [6]:
print(mntopts['shortid'])

booty


In [8]:
mntopts['shortid'] = 'F5'

In [9]:
target = mntopts['shortid']._apath.target
target.flush()

flushing /shortid <DictFS.DictFS object at 0x7f81f054dc70>
flushing slice(13, 18, None) user,shortid=F5,idcard=localuser
address is slice, service is Dcel
flushing /1/mntopts <DictFS.DictFS object at 0x7f81f056ca90>
flushing slice(53, 88, None) file://fs2.localhost/  {cburnuser}/example/  cburnfs user,shortid=F5,idcard=localuser 0 0
address is slice, service is Dcel
flushing slice(666, 758, None) # # Cosm / Etc / FSTab

# the .cosm/etc/fstab used by cburn is shared between hosts. The concept of 'localhost' is centric to a generic host model. A file url is relative to the generic host model, whereas a relative file path is relative to the working directory of cloudburner at runtime on each host.

# experimental: include a hostname in the 'file://' url to limit the scope of a filepath to a specific host.

# experimental: proxy the 'file' protocol and allow subdomain syntax to specify shares. The path component is relative to the share.

# idea: make filepaths relative to the fstab's loca

In [9]:
# Apaths
from json import dumps
from DcelJSONEncoder import DcelJSONEncoder

apath = mntopts._apath
while not apath is None:
    dcel = apath.target
    print(f"""{apath} {repr(dcel)}
    address: {dcel.address}
    service: {type(dcel.service)}
    lookup result: {repr(dcel.service.lookup(dcel.address))}
    value type: {type(dcel.value)}
    value: {dcel.value}
    json: {dumps(dcel,cls=DcelJSONEncoder)}
    """)
    print()
    apath = apath.parent
    

<APath.APath object at 0x7f2d501798b0> <Dcel.Dcel object at 0x7f2d50179a00>
    address: /
    service: <class 'DictFS.DictFS'>
    lookup result: {'user': <Dcel.Dcel object at 0x7f2d50179a60>, 'shortid': <Dcel.Dcel object at 0x7f2d50122190>, 'idcard': <Dcel.Dcel object at 0x7f2d50122a30>}
    value type: <class 'dict'>
    value: {'user': <Dcel.Dcel object at 0x7f2d50179a60>, 'shortid': <Dcel.Dcel object at 0x7f2d50122190>, 'idcard': <Dcel.Dcel object at 0x7f2d50122a30>}
    json: {"user": "", "shortid": "booty", "idcard": "localuser"}
    

<APath.APath object at 0x7f2d50179b80> <Dcel.Dcel object at 0x7f2d5010b550>
    address: /1/mntopts
    service: <class 'DictFS.DictFS'>
    lookup result: <Dcel.Dcel object at 0x7f2d5010b8b0>
    value type: <class 'str'>
    value: user,shortid=booty,idcard=localuser
    json: "user,shortid=booty,idcard=localuser"
    

<APath.APath object at 0x7f2d50179c10> <Dcel.Dcel object at 0x7f2d50111310>
    address: /1
    service: <class 'DictFS.DictFS'

In [12]:
print(mntopts['shortid'])
print(mntopts['shortid']._apath.target.address)
print(mntopts['shortid']._apath.target.service)
repr(mntopts['shortid']._apath.target.value)

shortid_dcel = mntopts['shortid']._apath.target.value
repr(shortid_dcel)
print(shortid_dcel.address)
repr(shortid_dcel.service)

print()
shortid_parent = shortid_dcel.service
print(shortid_parent._map)
print(shortid_parent.address)
print(shortid_parent.service)

print()
shortid_grandpa = shortid_parent.value
print(shortid_grandpa._map)
print(shortid_grandpa.address)
print(shortid_grandpa.service)

berry
/shortid
slice(13, 15, None)

None
/1/mntopts

None
slice(53, 85, None)
file://fs2.localhost/  {cburnuser}/example/  cburnfs user,shortid=berry,idcard=localuser 0 0


In [13]:
shortid_parent.flush()

flushing /1/mntopts <DictFS.DictFS object at 0x7f027c38da60>
target is Dcel
flushing slice(53, 85, None) file://fs2.localhost/  {cburnuser}/example/  cburnfs user,shortid=berry,idcard=localuser 0 0
address is slice, service is Dcel
flushing slice(666, 755, None) # # Cosm / Etc / FSTab

# the .cosm/etc/fstab used by cburn is shared between hosts. The concept of 'localhost' is centric to a generic host model. A file url is relative to the generic host model, whereas a relative file path is relative to the working directory of cloudburner at runtime on each host.

# experimental: include a hostname in the 'file://' url to limit the scope of a filepath to a specific host.

# experimental: proxy the 'file' protocol and allow subdomain syntax to specify shares. The path component is relative to the share.

# idea: make filepaths relative to the fstab's location, ie: for ./.cosm/etc/fstab the relative root is ../../../



file://fs2.localhost/  {cburnuser}/example/  cburnfs user,shortid=berry

In [53]:

from json import dumps
from DcelJSONEncoder import DcelJSONEncoder
print(mntopts)
print(dumps(mntopts._apath.target,cls=DcelJSONEncoder))
print('---')
print(dumps(mntopts._apath.parent.parent.target,cls=DcelJSONEncoder))


{'user': <Dcel.Dcel object at 0x7f84c857c580>, 'shortid': <Dcel.Dcel object at 0x7f84c857c610>, 'idcard': <Dcel.Dcel object at 0x7f84c857c430>}
{"user": "", "shortid": "bluegil", "idcard": "localusererile://fs.localhost  "}
---
{"spec": "dav://fs2.localhost", "file": "/home/raygan", "vfstype": "cburnfs", "mntopts": "user,shortid=bluegil,idcard=localusererile://fs.localhost  ", "freq": "burnuser}/example/  ", "passno": "urnfs u"}


In [9]:
mntopts._apath.target.flush()

flushing / <DictFS.DictFS object at 0x7f3f8413aeb0>


In [20]:
print(mntopts['shortid']._apath.parent.parent.target.address)

/1/mntopts


In [19]:
from json import dumps
from DcelJSONEncoder import DcelJSONEncoder
print(dumps(mntopts['shortid']._apath.parent.parent.target,cls=DcelJSONEncoder))

"user,shortid=SNOTGREEN,idcard=localuserer"


In [15]:
mntopts['shortid']._apath.parent.parent.target.flush()

flushing /1/mntopts <DictFS.DictFS object at 0x7f3f8415ae20>
target is Dcel
flushing slice(41, 79, None) dav://fs2.localhost /home/raygan cburnfs user,shortid=SNOTGREEN,idcard=localuserer
address is slice, service is Dcel
flushing slice(666, 745, None) # # Cosm / Etc / FSTab

# the .cosm/etc/fstab used by cburn is shared between hosts. The concept of 'localhost' is centric to a generic host model. A file url is relative to the generic host model, whereas a relative file path is relative to the working directory of cloudburner at runtime on each host.

# experimental: include a hostname in the 'file://' url to limit the scope of a filepath to a specific host.

# experimental: proxy the 'file' protocol and allow subdomain syntax to specify shares. The path component is relative to the share.

# idea: make filepaths relative to the fstab's location, ie: for ./.cosm/etc/fstab the relative root is ../../../



dav://fs2.localhost /home/raygan cburnfs user,shortid=SNOTGREEN,idcard=localusere

In [22]:
mntopts['shortid']._apath.target.flush()

flushing /shortid <DictFS.DictFS object at 0x7fca440974c0>
target is Dcel
flushing slice(13, 22, None) user,shortid=BLUE,idcard=localuser
address is slice, service is Dcel
flushing /1/mntopts <DictFS.DictFS object at 0x7fca44281fd0>
target is Dcel
flushing slice(41, 80, None) dav://fs2.localhost /home/raygan cburnfs user,shortid=HELLOELLO,idcard=localuser 0 
address is slice, service is Dcel
flushing slice(666, 749, None) # # Cosm / Etc / FSTab

# the .cosm/etc/fstab used by cburn is shared between hosts. The concept of 'localhost' is centric to a generic host model. A file url is relative to the generic host model, whereas a relative file path is relative to the working directory of cloudburner at runtime on each host.

# experimental: include a hostname in the 'file://' url to limit the scope of a filepath to a specific host.

# experimental: proxy the 'file' protocol and allow subdomain syntax to specify shares. The path component is relative to the share.

# idea: make filepaths re

In [1]:
print(mntopts._apath.target.value)
print(mntopts._apath.target.address)
print(mntopts._apath.target.service)
print(mntopts._apath.target.service.fsdict['shortid'].service.address)
print(mntopts._apath.target.service.fsdict['shortid'].service.service)
print(mntopts._apath.target.service.fsdict['shortid'].service.service.fsdict.keys())

addr  = mntopts._apath.target.service.fsdict['shortid'].service.address
svc   = mntopts._apath.target.service.fsdict['shortid'].service.service
res   = svc.lookup(addr)

print(type(res))
print(type(res.value))
print(res)


NameError: name 'mntopts' is not defined

In [21]:
print(mntopts._apath.target.address)
print(mntopts._apath.target.service)
print(mntopts._apath.target.service.fsdict)
print(mntopts._apath.target.service.fsdict["shortid"].address)
print(mntopts._apath.target.service.fsdict["shortid"].service)
print(mntopts._apath.target.service.fsdict["shortid"].service.address)
print(mntopts._apath.target.service.fsdict["shortid"].service.service)
print(mntopts._apath.target.service.fsdict["shortid"].service.service.fsdict)
print(mntopts._apath.target.service.fsdict["shortid"].service.service.fsdict['1']['mntopts'])
print(mntopts._apath.target.service.fsdict["shortid"].service.service.fsdict['1']['mntopts'].address)
print(mntopts._apath.target.service.fsdict["shortid"].service.service.fsdict['1']['mntopts'].service)
print(type(mntopts._apath.target.service.fsdict["shortid"].service.service.fsdict['1']['mntopts']))
print(mntopts._apath.target.service.fsdict["shortid"].service.service.fsdict['1']['mntopts'].service.address)
print(mntopts._apath.target.service.fsdict["shortid"].service.service.fsdict['1']['mntopts'].service.service)
print(type(mntopts._apath.target.service.fsdict["shortid"].service.service.fsdict['1']['mntopts'].service))


/
{'user': <Dcel.Dcel object at 0x7fca4421a100>, 'shortid': <Dcel.Dcel object at 0x7fca44106ee0>, 'idcard': <Dcel.Dcel object at 0x7fca44106b20>}
slice(13, 22, None)
user,shortid=BLUE,idcard=localuser
/1/mntopts
{'1': {'spec': <Dcel.Dcel object at 0x7fca44555a30>, 'file': <Dcel.Dcel object at 0x7fca44555d00>, 'vfstype': <Dcel.Dcel object at 0x7fca445559d0>, 'mntopts': <Dcel.Dcel object at 0x7fca45d63520>, 'freq': <Dcel.Dcel object at 0x7fca45d639a0>}, '2': {'spec': <Dcel.Dcel object at 0x7fca44499d30>, 'file': <Dcel.Dcel object at 0x7fca44499df0>, 'vfstype': <Dcel.Dcel object at 0x7fca44499e20>, 'mntopts': <Dcel.Dcel object at 0x7fca444d03d0>, 'freq': <Dcel.Dcel object at 0x7fca444d0250>}, '3': {'spec': <Dcel.Dcel object at 0x7fca444d0100>, 'file': <Dcel.Dcel object at 0x7fca442415e0>, 'vfstype': <Dcel.Dcel object at 0x7fca440dfca0>, 'mntopts': <Dcel.Dcel object at 0x7fca441b3340>, 'freq': <Dcel.Dcel object at 0x7fca44246be0>}, '4': {'spec': <Dcel.Dcel object at 0x7fca4449c220>, 'file'

In [18]:
# Debugging: This proves that something is not triggering a write via Fudge.
# This long statement works to flush the contents of the base object to storage.
mntopts._apath.target.service.fsdict["shortid"].service.service.fsdict['1']['mntopts'].service.service.value = mntopts._apath.target.service.fsdict["shortid"].service.service.fsdict['1']['mntopts'].service.service.value


In [1]:
print(mntopts["shortid"]._apath.target.service.fsdict['shortid'])


NameError: name 'mntopts' is not defined

In [56]:
mntopts = fu_root/'@/etc/fstabtest.fstab/1/mntopts.cskvp'
mntopts['idcard'] = 'raygan'
print(mntopts['idcard'])

raygan


In [52]:
print(mntopts['shortid'])
mntopts['shortid'] = "C"

f2


In [54]:
# Test writing multiple values to file at different positions.
entry = fu_root/'@/etc/fstabtest.fstab/1'
entry['file'] = '/home/raygan'
print(entry['file']._apath.target)
entry = fu_root/'@/etc/fstabtest.fstab/1'
entry['spec'] = 'dav://fs2.localhost'


/home/raygan


In [33]:
fstab = fu_root/'@/etc/fstabtest.fstab'

In [34]:
filefield = fstab/'1/file'
print(filefield)

/home/raygan


In [7]:
fstab['1/file'] = "cornfish"

In [8]:
from json import dumps
from DcelJSONEncoder import DcelJSONEncoder
print(dumps(fstab._apath.target,cls=DcelJSONEncoder))

{"1": {"spec": "file://fs2.localhost/", "file": "cornfish", "vfstype": "  cburn", "mntopts": "s user,shortid=f2,idcard=localus", "freq": "r", "passno": "0"}, "2": {"spec": "\nfile://fs.localhos", "file": " {cburnuser}/example", "vfstype": " cburnf", "mntopts": " user,shortid=fs,idcard=localuse", "freq": " ", "passno": " "}, "3": {"spec": "\nfile://fs3.localhost", "file": " {cburnuser}/example", "vfstype": " cburnf", "mntopts": " user,shortid=f3,idcard=localuse", "freq": " ", "passno": " "}, "4": {"spec": "\nfile://cburnwebui.localhost", "file": " {cburnuser}/example", "vfstype": " cburnf", "mntopts": " nouser,idcard=localuse", "freq": " ", "passno": " "}}


In [9]:

print(fu_root/'@/etc/fstabtest')

# # Cosm / Etc / FSTab

# the .cosm/etc/fstab used by cburn is shared between hosts. The concept of 'localhost' is centric to a generic host model. A file url is relative to the generic host model, whereas a relative file path is relative to the working directory of cloudburner at runtime on each host.

# experimental: include a hostname in the 'file://' url to limit the scope of a filepath to a specific host.

# experimental: proxy the 'file' protocol and allow subdomain syntax to specify shares. The path component is relative to the share.

# idea: make filepaths relative to the fstab's location, ie: for ./.cosm/etc/fstab the relative root is ../../../



file://fs2.localhost/  cornfish  cburnfs user,shortid=f2,idcard=localuser 0 0
file://fs.localhost    {cburnuser}/example/  cburnfs user,shortid=fs,idcard=localuser 0 0
file://fs3.localhost/  {cburnuser}/example/  cburnfs user,shortid=f3,idcard=localuser 0 0
file://cburnwebui.localhost/  {cburnuser}/example/  cburnfs nouser,idcard=lo

In [17]:
fstab = fu_root/'@/etc/fstab'
print(f"{repr(fstab._apath.target)} {repr(fstab._apath.target.address)}")
fstab_apath = APath(fstab)
print(f"{repr(fstab_apath.target)} {repr(fstab_apath.target.address)}")

entry_ref1 = fu_root/'@/etc/fstab/1'
entry_ref2 = fstab/'1'

print(f"{repr(entry_ref1._apath.target)}")
print(f"{repr(entry_ref2._apath.target)}")

print(entry_ref1)
print(entry_ref2)


<Dcel.Dcel object at 0x7feba529a3d0> '/@/etc/fstab'
<Dcel.Dcel object at 0x7feba529a3d0> '/@/etc/fstab'
{'spec': <Dcel.Dcel object at 0x7feba51af1f0>, 'file': <Dcel.Dcel object at 0x7feba51af4c0>, 'vfstype': <Dcel.Dcel object at 0x7feba51af160>, 'mntopts': <Dcel.Dcel object at 0x7feba51af490>}
{'spec': <Dcel.Dcel object at 0x7feba51af910>, 'file': <Dcel.Dcel object at 0x7feba51afd00>, 'vfstype': <Dcel.Dcel object at 0x7feba51afb80>, 'mntopts': <Dcel.Dcel object at 0x7feba51af550>}


In [6]:
print(f"{repr(fu_root['.@/etc/fstab']._apath.target)}")
print(f"{repr(fu_root['.@/etc/fstab']._apath.target.address)}")
for fu in fu_root['.@/etc']:
    print(f"""
    {repr(fu)}
    {repr(fu._apath)}
    {repr(fu._apath.target)}
    {repr(fu._apath.target.address)}
          """)


TypeError: None

In [75]:
x=fu/'@/etc/fstab'

for ea in x:
    print(f"{ea._apath.getinfo()} {repr((ea/'spec')._apath.target.value)}")
    
print(repr((x/'2/spec')._apath.target.value.service))

<dir '1'> <Dcel.Dcel object at 0x7f29adfe5250>
<dir '2'> <Dcel.Dcel object at 0x7f29adfbc310>
<dir '3'> <Dcel.Dcel object at 0x7f29ae003070>
<dir '4'> <Dcel.Dcel object at 0x7f29ae07d8e0>


In [76]:
a = APath(x/'2/spec')
a.target.value.value = "blue"
print(a.target.value)
print(x['1'])
print(repr(x._apath.target['1']))


blue
{'spec': <Dcel.Dcel object at 0x7f29ae4d0790>, 'file': <Dcel.Dcel object at 0x7f29ae4d06a0>, 'vfstype': <Dcel.Dcel object at 0x7f29ae4d0b20>, 'mntopts': <Dcel.Dcel object at 0x7f29ae521cd0>, 'freq': <Dcel.Dcel object at 0x7f29ae521be0>, 'passno': <Dcel.Dcel object at 0x7f29ae521d90>}


TypeError: None

In [51]:
from json import dumps
from DcelJSONEncoder import DcelJSONEncoder

In [18]:
print(x["2"])
print(dumps(x["2"]._apath.target,cls=DcelJSONEncoder))
x["2"]["spec"]._apath.target.value.value = "hello"
print(x["2"]["spec"]._apath.target.value.value)
print(dumps(x["2"]._apath.target,cls=DcelJSONEncoder))


{'spec': <Dcel.Dcel object at 0x7f29cc44ee80>, 'file': <Dcel.Dcel object at 0x7f29cc44ecd0>, 'vfstype': <Dcel.Dcel object at 0x7f29cc44ebb0>, 'mntopts': <Dcel.Dcel object at 0x7f29cc44eeb0>}
{"spec": "file://fs.localhost", "file": "{cburnuser}/example/", "vfstype": "cburnfs", "mntopts": "user,shortid=fs,idcard"}
file://fs.localhost
{"spec": "file://fs.localhost", "file": "{cburnuser}/example/", "vfstype": "cburnfs", "mntopts": "user,shortid=fs,idcard"}


In [28]:
from json import dumps
from DcelJSONEncoder import DcelJSONEncoder
target = (x/'2/spec')._apath.target
target.value = "sftp://fs.cburn.io"
target.flush()
print(target)
print(x/'2/spec')
print(dumps((x/'2/spec')._apath.target,cls=DcelJSONEncoder))

b'sftp://fs.cburn.io'
file://fs.localhost
"file://fs.localhost"


In [ ]:
fu = Fudge('file://fs.localhost')

In [132]:
fu['boats/skiff.txt'] = "My skiff is *somewhat* faster than your skull."

In [133]:
print(fu['boats/skiff.txt'])

My skiff is *somewhat* faster than your skull.


In [128]:
w = fu['@/etc/fstab'].fstab
x = w['1/spec']
print((x._apath.target.service.fsdict))
print(x._apath.target.address)

w['1/spec'] = 'file://fs200.localhost/'
print('---')
print(w)



KeyError: '@/etc/fstab'

#### Experiment to map the data from a dict (in the case of fstab) to the file stream.

Note: The data could just be formatted and written into the file. However, there may be comments or other non-parsed data in the original file which the parser ignored. Since that is much the case with the `etc/fstab` file, a byte-map needs to be kept to remap the new data to the file.

The example uses start, stop values, followed by a nested map.

    example_map = {'1':[0,200,nested_map_1],
                   '2':[201,300,nested_map_2]
                  }
    nested_map_1 = {'spec':[0,21,None],
                    'file':[22,20,None],
                    ...
                   }
    nested_map_2 = { ... }
etc.

On each write to the buffered data, a change log is made:

    change_log = {'1':{'spec':None, 'file':None}}
    
In the above example, `data['1']['spec']` and `data['1']['file']` changed.


To write the data back to the actual file a dirty bytes ordered dictionary is made:

lookup `example_map['1'][2] == 'spec'` and `example_map['1'][2] == 'file'`

From those, build the `dirty_bytes` ordered dictionary

    dirty_bytes = { 0: (old_len,new_len,new_data),
                    50: (old_len2,new_len2,new_data2),
                  }

oldfileposition = 0;

Copy the old file into new file until next `dirty_bytes` offset.
Then copy the dirty bytes data into the new file.
Advance the old file by `old_len`.
Repeat.


In [40]:
# dirty bytes { starting byte in old file: (old len, new len, new data) }
dirty_bytes = { 0: (21, 25, "new data"),
               50: (20, 30, "new data 2")
              }
print(dirty_bytes)

{0: (21, 25, 'new data'), 50: (20, 30, 'new data 2')}


In [18]:
# usecase: list and select a single
# entry from the fstab file.
#

fu = Fudge('file://fs.localhost')

x=fu/'.cosm/etc/fstab'

# list fstab

for ea in x:
    print(ea._apath.getinfo())
    
# select target

x=x/'1'

print('is_dir: '+str(x._apath.isdir()))
print('is_glob: '+str(hasattr(x._apath,"_isglob")))
if(x._apath.isdir()):
    for ea in x:
        print(ea._apath.getinfo())
else:
    for ea in x:
        print(ea)

<dir '1'>
<dir '2'>
<dir '3'>
<dir '4'>
is_dir: True
is_glob: False
<file 'spec'>
<file 'file'>
<file 'vfstype'>
<file 'mntopts'>
<file 'freq'>
<file 'passno'>


In [4]:
fu = Fudge('file://fs.localhost')

x=fu/'.cosm/etc/fstab/*/spec.url/netloc'
for e in x:
    a = e._apath
    i = a.getinfo().raw
    if hasattr(x._apath,('_isglob')):
        i['basic']['name'] = str(e)
    print(i)

x=fu['.cosm']['etc']['fstab']['*']['spec']
for e in x:
    print(e.url['netloc'])

{'basic': {'name': 'fs2.localhost', 'is_dir': False}}
{'basic': {'name': 'fs.localhost', 'is_dir': False}}
{'basic': {'name': 'fs3.localhost', 'is_dir': False}}
{'basic': {'name': 'cburnwebui.localhost', 'is_dir': False}}
fs2.localhost
fs.localhost
fs3.localhost
cburnwebui.localhost


In [5]:
fstab=fu/'.cosm/etc/fstab'
print(fstab['*']['spec']._apath.isdir())

for ea in fstab['*']['spec']:
    print(ea)

True
file://fs2.localhost/
file://fs.localhost
file://fs3.localhost/
file://cburnwebui.localhost/


In [6]:
print(fstab/'*'/'spec.url')
for ea in fstab/'*'/'spec.url':
    print(ea/'netloc')
    

[<__main__.Fudge object at 0x11e049f40>, <__main__.Fudge object at 0x11e0904f0>, <__main__.Fudge object at 0x11e090a60>, <__main__.Fudge object at 0x11e090a30>]
fs2.localhost
fs.localhost
fs3.localhost
cburnwebui.localhost


In [7]:
print(splitexists(fu,'.cosm/etc','/'))
print(splitexists(fu/'.cosm/etc','fstab.json','.'))

('.cosm/etc', [])
('fstab', ['json'])


In [8]:
path = '.cosm/etc/fstab/*/spec.url/netloc'
res = fu/path
for ea in res:
    print(ea)

fs2.localhost
fs.localhost
fs3.localhost
cburnwebui.localhost


In [12]:
hosts = [str(e) for e in fu/'.cosm/etc/fstab/*/spec.url/netloc']
sources = [str(e) for e in fu/'.cosm/etc/fstab/*/spec']
vfstypes = {str(e) for e in fu/'.cosm/etc/fstab/*/vfstype'}

In [11]:
print(hosts)
print(sources)
print(vfstypes)

[<__main__.Fudge object at 0x11c955670>, <__main__.Fudge object at 0x11c955b50>, <__main__.Fudge object at 0x11ed0c3a0>, <__main__.Fudge object at 0x11ed0c4c0>]
[<__main__.Fudge object at 0x11ed0c880>, <__main__.Fudge object at 0x11ed0cb20>, <__main__.Fudge object at 0x11ed0cc40>, <__main__.Fudge object at 0x11ed0cd60>]
{'cburnfs'}


In [19]:
def dir_magic(self):
        if self._apath.isdir():
            return self
        else:
            try:
                addr = self._apath.target.address
                typename = basename(addr)
                res = self.__getattr__(typename)
                if res._apath.isdir():
                    return res
            except:
                pass
        raise NotADirectoryError
            
a = fu/'.cosm/etc/'

b = dir_magic(a)
for ea in b['fstab']['1']:
    print(ea)

spec
file
vfstype
mntopts
freq
passno


In [20]:
a=[1,2,3]
b=iter(a)
for ea in b:
    print(ea)
    break
print('---')
c = [*b]
for ea in c:
    print(ea)
print(c)
print(b)
for ea in b:
    print(ea)

1
---
2
3
[2, 3]


In [21]:
path = '.cosm/etc/fstab.fstab/1/spec'
a = fu['.cosm/etc']
b,segs = splitexists(fu,path,'/')
print(b)
print(segs)
c,segs = splitexists(fu[b],segs[0],'.')
print(c)
print(segs)
d = fu[b][c].__getattr__(segs[0])
print(d)


.cosm/etc
['fstab.fstab', '1', 'spec']
fstab
['fstab']
{'1': {'spec': 'file://fs2.localhost/', 'file': '/', 'vfstype': 'cburnfs', 'mntopts': 'user', 'freq': '0', 'passno': '0'}, '2': {'spec': 'file://fs.localhost', 'file': '/', 'vfstype': 'cburnfs', 'mntopts': 'user', 'freq': '0', 'passno': '0'}}


In [24]:
f = fu['/']['.cosm']['etc']['fstab']
g = f.fstab
h = g._apath.target.value['1']
h = g['1']['spec']

In [29]:
type(g._apath.target.value)
print(h)


file://fs2.localhost/


In [3]:
path = '.cosm/etc/fstab'

a = fu[path].fstab['1']['spec'].url['netloc']
print(type(a))
print(a)

#b = fu/'.cosm/etc/fstab/1/spec.url/netloc'
#print(b)

NameError: name 'fu' is not defined

In [8]:
ar = ['1','2','3']
fg = Fudge(ar)
print(fg)
for ea in fg:
    print(ea)

['1', '2', '3']
1
2
3


In [9]:
name = 'a.b.c'
exts = name.split('.',1)
while exts != [name]:
    ext = exts[1]
    name = exts[0]
    exts = name.split('.',-1)
name = exts[0]
print(name)

a


In [10]:
b = fu['.cosm/etc/fstab']
for ea in b:
    if b/ea/'vfstype' == 'cburnfs':
        print(str(b/ea/'spec'))
    

/.cosm/etc/fstab
last seg...
1
/.cosm/etc/fstab
enter fudgedot
seg: 1
tryseg: 1


AttributeError: 'NoneType' object has no attribute '_apath'

In [29]:
name = ""
try:
    x
except NameError as exception:
    name = (str(exception). 
          removeprefix("name '"). 
          removesuffix("' is not defined"))
print(name)

x


In [3]:
print(fu['.cosm']['etc']['fstab']/'1')

{'spec': 'file://fs2.localhost/', 'file': '/', 'vfstype': 'cburnfs', 'mntopts': 'user', 'freq': '0', 'passno': '0'}


In [12]:
fu/*


SyntaxError: invalid syntax (<ipython-input-12-ce50f3fcbe94>, line 1)

In [18]:
addr = fu['.cosm/etc']['fstab']._apath.target.address
fu._apath.isdir(addr)

False

In [4]:
fu = Fudge(APath('fs','file'))


print(fu/'a'/'b'/'c'@'hello') # ugly
print((fu/'a'/'b'/'c').hello) # clumsy
print(fu.a.b.c/'hello') # misleading
print(fu.a.b.c@'hello') # misleading
print(fu['a/b/c.hello']) # confusing
print(fu('a/b/c.hello')) # func-like
print(fu/'a/b/c'*'hello') # weird
print(fu/'a/b/c'/'this'@'hello') # weird
print(fu/'a/b/c.hello') # streamlined

# this form makes the most sense
# ie, three branches, attribute
print(fu/'a/b/c.hello')

# equivalent alternative
print(fu['a']['b']['c'].hello) # ugly

# three surfaces
print( fu['a']('twelve').six )

# a table
fu['a/b/c.transf']._table('spec','vfstype')

# bracket syntax

fu['child'].typeshift('args'){['@attr']}

SyntaxError: invalid syntax (<ipython-input-4-a193f3076e34>, line 29)

In [6]:
fu@'hello'


NameError: name 'fu' is not defined

In [6]:
class A():
    def __call__(self):
        return 2
    
a = A()

print(a)
a()

2

In [7]:
fu('fstab').entry[1].vfstype

fu.fstab('some','args').entry[1].vfstype
fu.fstab['fs1.raygan.com'].vfstype
x = fu.fstab(encoding='utf-8')['fs1.raygan.com'].vfstype
print(x)

fudge


In [6]:
type(fu)
dir(fu)
fu.__class__
print((fu/'a'/'b'/'c').hello)

class Fidge():
    def __truediv__(self, other):
        return Fidge()
    
    def __getitem__(self, item):
        return Fidge()
    
    def __matmul__(self, item):
        return Fidge()
    
    hello = 'rock on' 
    
import __main__

fu.__class__ = __main__.Fidge
print(fu['a/b/c'].hello)

hi
rock on


In [7]:


def __getattr__(name):
    if name == 'hello':
        return 'hi'
    return name


#dir(__main__.__builtin__)
#type(__main__)

#hello = 'hey'

import runtimeLocalsTest

hi


In [8]:
from __main__ import bruce, dan, kelly

print(bruce)
print(dan)
print(kelly)
# print(bob)

bruce
dan
kelly


In [10]:
# you can only decorate functions and methods

def fake(varname):
    def wrapper():
        from __main__ import varname
    return wrapper

#@fake
#bruce
yy
fake('me')()
varname


NameError: name 'varname' is not defined

In [2]:
class FudgeIter():
    def __init__(self, fudge):
        self.fudge = fudge
        self._iter = iter(fudge._apath.listdir())
    def __iter

SyntaxError: invalid syntax (<ipython-input-2-a7df1eec3a13>, line 5)